<a href="https://colab.research.google.com/github/dmamur/elembert/blob/noPosEmbeddings/element_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re, glob,os,sys,pickle,random
from collections import defaultdict

In [2]:
!pip install ase
!git clone -b noPosEmbeddings https://github.com/dmamur/elembert.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 15.1 MB/s eta 0:00:00
Cloning into 'elembert'...
remote: Enumerating objects: 1490, done.
remote: Counting objects: 100% (572/572), done.
remote: Compressing objects: 100% (372/372), done.
remote: Total 1490 (delta 362), reused 340 (delta 195), pack-reused 918 (from 1)
Receiving objects: 100% (1490/1490), 120.47 MiB | 18.90 MiB/s, done.
Resolving deltas: 100% (821/821), done.
Updating files: 100% (389/389), done.


# Load data from matbench

In [6]:
from ase.io import read,write
from scipy.ndimage import gaussian_filter1d
from ase.neighborlist import neighbor_list
from joblib import dump, load
from sklearn.cluster import KMeans
from sklearn import decomposition
from ase import Atoms

ellst=open('/content/elembert/models/pcakm/ellist.txt','r').read().split('\n')
print(ellst)

km = {i: load('/content/elembert/models/kmeans/'+i+'_kmeans.pkl') for i in ellst}

def getRawInputs(types,atoms,x,v):
    i, d = neighbor_list('id', atoms, 10.0, self_interaction=False)
    rdfatoms,ntypes=[],[]
    for k,l in enumerate(atoms):
        el=types[k]
        y = np.zeros(100)
        dist = np.round(d[i==k]*10)
        a,b=np.unique(dist, return_counts=True)
        np.put(y,a.astype(int)-1,b)
        values=gaussian_filter1d(y/v,1)
        num = km[el].predict(np.nan_to_num([values],nan=0,posinf=0, neginf=0))[0]
        ntypes.append(el+str(num))#el2id[el+str(num)]
    return ntypes

['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Rn']


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator KMeans from version 1.4.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
from ase.build import molecule
atoms = molecule('CH3CH2NH2')
#typesLstTest = getModelInputs(train_inputs)

In [8]:
x= np.arange(0,10,0.1)
v = np.concatenate([[1],4*np.pi/3*(x[1:]**3 - x[:-1]**3)])
typesN=getRawInputs(atoms.get_chemical_symbols(),atoms,x,v)

In [9]:
print(typesN)

['C5', 'C3', 'N1', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0']


In [10]:
from ase.io import read, write

In [13]:
atoms=read('/content/elembert/models/pcakm/Figure2a.xyz',format='xyz')
typesN1=getRawInputs(atoms.get_chemical_symbols(),atoms,x,v)
print(typesN1)

['C3', 'C1', 'C6', 'C6', 'S7', 'C1', 'C6', 'C3', 'N1', 'C6', 'C3', 'S7', 'C6', 'C1', 'O0', 'O0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0']


In [14]:
atoms=read('/content/elembert/models/pcakm/Figure2b.xyz',format='xyz')
typesN2=getRawInputs(atoms.get_chemical_symbols(),atoms,x,v)
print(typesN2)

['Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Co3', 'Co3', 'O0', 'O0', 'O0', 'O0', 'O0', 'O0', 'O0', 'O0', 'O0', 'O0', 'O0', 'O0']
